In [9]:
import pandas as pd
import data_cleaning

time_attrs = [
    'pickup_datetime',
    'dropoff_datetime',
]

taxi = pd.read_csv('data/train.csv', parse_dates=time_attrs)
taxi_cleaned = data_cleaning.pipeline.fit_transform(taxi)
taxi_labels = taxi['trip_duration'].copy()
taxi_cleaned_pd = pd.DataFrame({
    'pickup_datetime': taxi_cleaned[:, 0],
    'dropoff_datetime': taxi_cleaned[:, 1],
    'passenger_count': taxi_cleaned[:, 2],
    'pickup_longitude': taxi_cleaned[:, 3],
    'pickup_latitude': taxi_cleaned[:, 4],
    'dropoff_longitude': taxi_cleaned[:, 5],
    'dropoff_latitude': taxi_cleaned[:, 6],
    'distance': taxi_cleaned[:, 7],    
})

In [10]:
taxi_cleaned_pd.describe()

,distance,dropoff_datetime,dropoff_latitude,dropoff_longitude,passenger_count,pickup_datetime,pickup_latitude,pickup_longitude
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.107602e-17,-1.205094e-14,1.014493e-13,3.885016e-14,-6.459772e-17,-2.639530e-14,3.492914e-15,-3.314967e-14
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-8.008460e-01,-1.773205e+00,-2.387999e+02,-6.789027e+02,-1.266532e+00,-1.773048e+00,-1.943734e+02,-6.764262e+02
25%,-5.141414e-01,-8.476855e-01,-4.434273e-01,-2.535194e-01,-5.056372e-01,-8.477397e-01,-4.128277e-01,-2.592424e-01
50%,-3.135425e-01,5.459914e-03,7.591738e-02,-8.968501e-02,-5.056372e-01,5.466079e-03,9.670856e-02,-1.164505e-01
75%,1.011218e-01,8.481785e-01,5.018093e-01,1.472646e-01,2.552578e-01,8.482028e-01,5.303712e-01,8.681542e-02
max,2.880152e+02,1.775168e+00,8.830261e+01,1.788973e+02,5.581523e+00,1.756776e+00,3.384965e+02,1.782458e+02


In [11]:
taxi_cleaned_pd_label = taxi_cleaned_pd.copy()
taxi_cleaned_pd_label['trip_duration'] = taxi['trip_duration']

In [12]:
taxi_cleaned_pd_label.describe()

,distance,dropoff_datetime,dropoff_latitude,dropoff_longitude,passenger_count,pickup_datetime,pickup_latitude,pickup_longitude,trip_duration
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.107602e-17,-1.205094e-14,1.014493e-13,3.885016e-14,-6.459772e-17,-2.639530e-14,3.492914e-15,-3.314967e-14,9.594923e+02
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.237432e+03
min,-8.008460e-01,-1.773205e+00,-2.387999e+02,-6.789027e+02,-1.266532e+00,-1.773048e+00,-1.943734e+02,-6.764262e+02,1.000000e+00
25%,-5.141414e-01,-8.476855e-01,-4.434273e-01,-2.535194e-01,-5.056372e-01,-8.477397e-01,-4.128277e-01,-2.592424e-01,3.970000e+02
50%,-3.135425e-01,5.459914e-03,7.591738e-02,-8.968501e-02,-5.056372e-01,5.466079e-03,9.670856e-02,-1.164505e-01,6.620000e+02
75%,1.011218e-01,8.481785e-01,5.018093e-01,1.472646e-01,2.552578e-01,8.482028e-01,5.303712e-01,8.681542e-02,1.075000e+03
max,2.880152e+02,1.775168e+00,8.830261e+01,1.788973e+02,5.581523e+00,1.756776e+00,3.384965e+02,1.782458e+02,3.526282e+06


In [13]:
corr_matrix = taxi_cleaned_pd_label.corr()
corr_matrix["trip_duration"].sort_values(ascending=False)

trip_duration        1.000000
distance             0.094777
pickup_longitude     0.026542
dropoff_longitude    0.014678
passenger_count      0.008471
dropoff_datetime     0.007819
pickup_datetime      0.006644
dropoff_latitude    -0.020677
pickup_latitude     -0.029204
Name: trip_duration, dtype: float64

In [17]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
import numpy as np

tree_reg = DecisionTreeRegressor()
tree_reg.fit(taxi_cleaned, taxi_labels)
tree_predictions = tree_reg.predict(taxi_cleaned)
tree_mse = mean_squared_error(taxi_labels, tree_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

In [18]:
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    rmse_scores = np.sqrt(-scores)
    print("Scores:", rmse_scores)
    print("Mean:", rmse_scores.mean())
    print("Standard deviation:", rmse_scores.std())
    
tree_reg = DecisionTreeRegressor()
tree_scores = cross_val_score(tree_reg, taxi_cleaned, taxi_labels,
                              scoring="neg_mean_squared_error", cv=10)
display_scores(tree_scores)

Scores: [  4412.82583894   4299.74111742   6714.11620977   4208.50937834
   7022.34362978   4438.05889609  11539.14918485   4385.91554089
   6700.14671706   4507.71951653]
Mean: 5822.85260297
Standard deviation: 2197.8395301


In [19]:
from sklearn.model_selection import GridSearchCV

param_grid =[
    {'max_depth': [3, 5, 10]},
]

tree_reg = DecisionTreeRegressor()
tree_grid_search = GridSearchCV(tree_reg, param_grid, cv=5, scoring='neg_mean_squared_error')

tree_grid_search.fit(taxi_cleaned, taxi_labels)

tree_grid_search.best_params_

{'max_depth': 5}

In [20]:
cvres = tree_grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

5208.92988043 {'max_depth': 3}
5207.40653419 {'max_depth': 5}
5474.82326864 {'max_depth': 10}
